# Six Nations Tournament Problem

# Code from the open-source library on Artificial Intelligence: A Modern Approach

The code can be found [here](https://github.com/aimacode/aima-python).

In [1]:
%pip install utils
import sys
from collections import deque
from utils import *

class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=0aaabc95781d3145c77b5f5d57300e0dd43c445aa192df6dd80b3f0cfe325abf
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [2]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [3]:
def depth_first_tree_search(problem):
    """
    [Figure 3.7]
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """

    frontier = [Node(problem.initial)]  # Stack

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None

In [4]:
def breadth_first_tree_search(problem):
    """
    [Figure 3.7]
    Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """

    frontier = deque([Node(problem.initial)])  # FIFO queue

    while frontier:
        node = frontier.popleft()
        if problem.goal_test(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None

In [5]:
import functools
import heapq

def memoize(func):
    cache = {}
    @functools.wraps(func)
    def memoized_func(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return memoized_func

class PriorityQueue:
    def __init__(self, order='min', priority_func=None):
        self.heap = []
        self.order = order
        self.priority_func = priority_func or (lambda x: x)

    def append(self, item):
        heapq.heappush(self.heap, (self.priority_func(item), item))

    def pop(self):
        if self.order == 'min':
            return heapq.heappop(self.heap)[1]
        elif self.order == 'max':
            return heapq.heappop(self.heap)[1]
        else:
            raise ValueError("Invalid order type. Use 'min' or 'max'.")

    def __contains__(self, item):
        return any(item == pair[1] for pair in self.heap)

    def __getitem__(self, key):
        for _, item in self.heap:
            if item == key:
                return item
        raise KeyError("Key not found in the priority queue.")

    def __delitem__(self, key):
        for i, (priority, item) in enumerate(self.heap):
            if item == key:
                self.heap.pop(i)
                heapq.heapify(self.heap)
                return
        raise KeyError("Key not found in the priority queue.")

    def __len__(self):
        return len(self.heap)


In [6]:
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f)
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

In [7]:
def uniform_cost_search(problem, display=False):
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda node: node.path_cost, display)

In [8]:
def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h)
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

# Modelling

We define a state as follows:


*   as a $n ⋅ m$ matrix, where $n$ represents rounds and $m$ matches at a round :
\begin{pmatrix}
m_{00} & m_{01} \\
m_{10} & m_{21} \\
m_{20} & m_{21} \\
\end{pmatrix}
where ∀ n ∈ R, ∀ m ∈ M, ∀ k ∈ {0, 1}, ∀ i ∀ j ∈ N (i $\neq$ j) $m_{nmk}$ $=$ $i$ ⊕ $m_{nmk}$ $=$ $j$.

The states will be as follows: ($m_{00}$, $m_{01}$,..., $m_{mn}$).

Initially, ∀ n ∈ R, ∀ m ∈ M, ∀ k ∈ {0, 1}, $m_{nm}$ = $-1$.

The goal test is as follows:
* ∀ i ∈ R, ∀ j ∈ M, $m_{i, j}$ $\neq$ $-1$, i.e. an empty slot, \\
and ∀ $m, m'$ ∈ M, (m $\neq$ m') and ∀ n ∈ N, n ∈ $m_{i, m}$ →  n ∉ $m_{i, m}$



# N Nations Without Constraint Implementation

Here, the action (or successor function) considers not to put a match that exists—i.e. ∀ r, r' ∈ R, ∀ m, m' ∈ M, ∀ i ∀ j ∈ N, ($i \neq j$ ∧ $m \neq m'$) ∃ $m_{rm} = (i, j)$ → ∄ ($m_{r'm'} = (i, j) ∧ m_{r'm'} = (j, i)$)—and ∀ r ∈ R, ∀ m, m' ∈ M ∀ n ∈ N, ∃ n ∈ $m_{rm}$ → n ∉ $m_{rm'}$.

The other constraints are not added.

In [19]:
class NNationsTournamentWithoutConstraint(Problem):
    def __init__(self, teams):
        self.teams = teams
        self.matches = self.generate_matches(self.teams)
        self.initial = tuple([(None, None)] * ((len(self.matches)//2)))

        Problem.__init__(self, self.initial)

    def generate_matches(self, teams):
        matches = []
        for i in range(len(teams) - 1):
            for j in range(i + 1, len(teams)):
                matches.append((teams[i], teams[j]))
                matches.append((teams[j], teams[i]))
        return matches

    def actions(self, state):
        if state[-1] != (None, None):
            return []
        else:
            m_index = state.index((None, None))
            round = m_index // (len(self.teams)//2)
            return [m for m in self.matches
                    if not (self.scheduling_conflicted(state, round, m_index, m)
                    or self.asymmetry_conflicted(state, m_index, m))
                    ]

    def scheduling_conflicted(self, state, round, m_index, m):
      return any(self.scheduling_conflict(m, state[n])
                  for n in range(len(state))
                  if (state[n] != (None, None)
                  and ((n//(len(self.teams)//2)) == round)
                  and n != m_index))

    def scheduling_conflict(self, m, n):
      return (m[0] == n[0]
              or m[0] == n[1]
              or m[1] == n[0]
              or m[1] == n[1])

    def asymmetry_conflicted(self, state, m_index, m):
      """Would placing a match m cause a violation of the natural constraint of symmetry?"""
      return any(self.asymmetry_conflict(m, state[n])
                 for n in range(len(state))
                 if n != m_index)

    def asymmetry_conflict(self, m, n):
      """Would placing a match m and n conflict with the natural symmetry constraint?"""
      return (n == m or n[::-1] == m)

    def result(self, state, action):
        new_match = state.index((None, None))
        new_state = list(state[:])
        new_state[new_match] = action
        return tuple(new_state)

    def goal_test(self, state):
      if state[-1] is (None, None):
        return False
      return not (any(self.asymmetry_conflicted(state, state.index(m), m)
                      for m in state)
                  or any(self.scheduling_conflicted(state, (m//(len(self.teams)//2)), m, state[m])
                      for m in range(len(state)))
                  )


    def value(self, state):
        return 0  # This is not an optimization problem, so value is not relevant

    def path_cost(self, c, state1, action, state2):
        return c + 1  # Cost is incremented by 1 for each step

    def h(self, node):
        """Return number of conflicting match placements for a given node"""
        num_conflicts = 0
        for (m_index, m) in enumerate(node.state):
            for (n_index, n) in enumerate(node.state):
                if n_index != m_index:
                    num_conflicts += self.asymmetry_conflict(m, n)
                if (n_index//(len(self.teams)//2)) == (m_index//(len(self.teams)//2)) and n_index != m_index: #Check if n, m are in the same round
                    num_conflicts += self.scheduling_conflict(m, n)

        return num_conflicts


<>:58: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:58: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-19-02d6d017792b>:58: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if state[-1] is (None, None):


In [20]:
six_nations_without_constraint = NNationsTournamentWithoutConstraint(("IT", "FR", "EN", "WA", "SC", "EI"))
four_nations_without_constraint = NNationsTournamentWithoutConstraint(("WA", "FR", "EN", "IT"))
eight_nations_without_constraint = NNationsTournamentWithoutConstraint(("IT", "FR", "EN", "WA", "SC", "EI", "ZA", "AU"))

In [ ]:
%%timeit
depth_first_tree_search(four_nations_without_constraint)

270 µs ± 6.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit
depth_first_tree_search(six_nations_without_constraint)

3.7 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
print(depth_first_tree_search(four_nations_without_constraint).solution())

[('IT', 'EN'), ('FR', 'WA'), ('IT', 'FR'), ('EN', 'WA'), ('EN', 'FR'), ('IT', 'WA')]


In [ ]:
print(depth_first_tree_search(six_nations_without_constraint).solution())

[('EI', 'SC'), ('WA', 'EN'), ('FR', 'IT'), ('EI', 'WA'), ('SC', 'FR'), ('EN', 'IT'), ('SC', 'WA'), ('EN', 'FR'), ('EI', 'IT'), ('EI', 'EN'), ('WA', 'FR'), ('SC', 'IT'), ('SC', 'EN'), ('EI', 'FR'), ('WA', 'IT')]


In [ ]:
%%timeit
breadth_first_tree_search(four_nations_without_constraint)

98.7 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
breadth_first_tree_search(six_nations_without_constraint)

KeyboardInterrupt: 

In [ ]:
print(breadth_first_tree_search(four_nations_without_constraint).solution())

[('IT', 'FR'), ('EN', 'WA'), ('IT', 'EN'), ('FR', 'WA'), ('IT', 'WA'), ('FR', 'EN')]


In [ ]:
print(breadth_first_tree_search(six_nations_without_constraint))

In [33]:
import time
s = time.time()
for i in range(10):
  astar_search(four_nations_without_constraint)
e = time.time()
(e - s)/10

0.00237421989440918

In [32]:
import time
s = time.time()
for i in range(10):
  astar_search(six_nations_without_constraint)
e = time.time()
(e - s)/10

0.027591967582702638

In [31]:
import time
s = time.time()
for i in range(10):
  astar_search(eight_nations_without_constraint)
e = time.time()
(e - s)/10

0.22528519630432128

In [27]:
print(astar_search(eight_nations_without_constraint))

<Node (('AU', 'EI'), ('EN', 'FR'), ('IT', 'SC'), ('WA', 'ZA'), ('AU', 'EN'), ('EI', 'FR'), ('IT', 'WA'), ('SC', 'ZA'), ('AU', 'FR'), ('EI', 'EN'), ('IT', 'ZA'), ('SC', 'WA'), ('AU', 'IT'), ('EI', 'SC'), ('EN', 'WA'), ('FR', 'ZA'), ('AU', 'SC'), ('EI', 'IT'), ('EN', 'ZA'), ('FR', 'WA'), ('AU', 'WA'), ('EI', 'ZA'), ('EN', 'IT'), ('FR', 'SC'), ('AU', 'ZA'), ('EI', 'WA'), ('EN', 'SC'), ('FR', 'IT'))>


# N Nations Tournaments With Constraints Implementation

Here, we add the following constraints:
1.   Constraint 1: No team cannot play consecutive matches at home/away. ∀ r, r' ∈ R (r $=$ r' - 1), ∀ m, m' ∈ M, (m < r⋅|N/2| + 1 and m' < r'⋅|N/2|), l, l' ∈ {0,1}, (l $\neq$ l'), ∀ n ∈ N, n ∈ $m_{rml}$ → n ∈ $m_{r'm'l'}$
2.   Constraint 2: On the last round, one match must be ENG v. FRA or FRA v. ENG.
3. (If constraint 1 does not work) Constraint 3: No team plays against ITA $\vee$ FRA away twice in consecutive matches.



# Implementation with Constraint 1

In [9]:
class NNationsTournamentWithLocationConstraint(Problem):
    def __init__(self, teams):
        self.teams = teams
        self.matches = self.generate_matches(self.teams)
        self.initial = tuple([(None, None)] * ((len(self.matches)//2)))

        Problem.__init__(self, self.initial)

    def generate_matches(self, teams):
        matches = []
        for i in range(len(teams) - 1):
            for j in range(i + 1, len(teams)):
                matches.append((teams[i], teams[j]))
                matches.append((teams[j], teams[i]))
        return matches

    def actions(self, state):
        if state[-1] != (None, None):
            return []
        else:
            m_index = state.index((None, None))
            round = m_index // (len(self.teams)//2)
            return [m for m in self.matches
                    if not (self.asymmetry_conflicted(state, m_index, m)
                    or self.location_conflicted(state, round, m)
                    )
                    ]

    def scheduling_conflicted(self, state, round, m_index, m):
      return any(self.scheduling_conflict(m, state[n])
                  for n in range(len(state))
                  if (state[n] != (None, None)
                  and ((n//(len(self.teams)//2)) == round)
                  and n != m_index))

    def scheduling_conflict(self, m, n):
      return (m[0] == n[0]
              or m[0] == n[1]
              or m[1] == n[0]
              or m[1] == n[1])

    def asymmetry_conflicted(self, state, m_index, m):
      return any(state[n] == m or state[n][::-1] == m
                 for n in range(len(state))
                 if n != m_index)

    def location_conflicted(self, state, round, m):
      return any(self.location_conflict(m, state[n])
                  for n in range((round-1)*(len(self.teams)//2), round*(len(self.teams)//2))
                )

    def location_conflict(self, m, n):
      return (m[0] == n[0] or m[1] == n[1])

    def result(self, state, action):
        new_match = state.index((None, None))
        new_state = list(state[:])
        new_state[new_match] = action
        return tuple(new_state)

    def goal_test(self, state):
      if state[-1] is (None, None):
        return False
      return not (any(self.asymmetry_conflicted(state, state.index(m), m)
                      for m in state)
                  # or any(self.scheduling_conflicted(state, (m//(len(self.teams)//2)), m, state[m])
                  #     for m in range(len(state)))
                  or any(self.location_conflicted(state, (m//(len(self.teams)//2)), state[m])
                          for m in range(len(state)))
                  )


    def value(self, state):
        return 0  # This is not an optimization problem, so value is not relevant

    def path_cost(self, c, state1, action, state2):
        return c + 1  # Cost is incremented by 1 for each step


<>:62: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:62: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-9-3664bfad455e>:62: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if state[-1] is (None, None):


In [10]:
six_nations_with_location_constraint = NNationsTournamentWithLocationConstraint(("IT", "FR", "EN", "WA", "SC", "EI"))
four_nations_with_location_constraint = NNationsTournamentWithLocationConstraint(("IT", "FR", "EN", "WA"))

In [ ]:
%%timeit
depth_first_tree_search(four_nations_with_location_constraint)

6.04 ms ± 503 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
print(depth_first_tree_search(four_nations_with_location_constraint))

None


In [ ]:
%%timeit
breadth_first_tree_search(four_nations_with_location_constraint)

6.23 ms ± 596 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
print(breadth_first_tree_search(four_nations_with_location_constraint))

None


# Implementation with constraint 2

In [ ]:
class NNationsTournamentWithFinalMatchConstraint(Problem):
    def __init__(self, teams):
        self.teams = teams
        self.matches = self.generate_matches(self.teams)
        self.initial = tuple([(None, None)] * ((len(self.matches)//2)))

        Problem.__init__(self, self.initial)

    def generate_matches(self, teams):
        matches = []
        for i in range(len(teams) - 1):
            for j in range(i + 1, len(teams)):
                matches.append((teams[i], teams[j]))
                matches.append((teams[j], teams[i]))
        return matches

    def actions(self, state):
        if state[-1] != (None, None):
            return []
        else:
            m_index = state.index((None, None))
            round = m_index // (len(self.teams)//2)
            return [m for m in self.matches
                    if not (self.scheduling_conflicted(state, round, m_index, m)
                    or self.asymmetry_conflicted(state, m_index, m)
                    )
                    ]

    def scheduling_conflicted(self, state, round, m_index, m):
      return any(self.scheduling_conflict(m, state[n])
                  for n in range(len(state))
                  if (state[n] != (None, None)
                  and ((n//(len(self.teams)//2)) == round)
                  and n != m_index))

    def scheduling_conflict(self, m, n):
      return (m[0] == n[0]
              or m[0] == n[1]
              or m[1] == n[0]
              or m[1] == n[1])

    def asymmetry_conflicted(self, state, m_index, m):
      return any(state[n] == m or state[n][::-1] == m
                 for n in range(len(state))
                 if n != m_index)

    def final_match_conflicted(self, state):
      return all(("EN", "FR") != state[n] and ("FR", "EN") != state[n]
                 for n in range(len(state)-(len(self.teams)//2), len(state))
                 if state[n] != (None, None)
                 )

    def location_conflicted(self, state, round, m):
      return any(self.location_conflict(m, state[n])
                  for n in range((round-1)*(len(self.teams)//2), round*(len(self.teams)//2))
                )

    def location_conflict(self, m, n):
      return (m[0] == n[0] or m[1] == n[1])

    def result(self, state, action):
        new_match = state.index((None, None))
        new_state = list(state[:])
        new_state[new_match] = action
        return tuple(new_state)

    def goal_test(self, state):
      if state[-1] is (None, None):
        return False
      return not (any(self.asymmetry_conflicted(state, state.index(m), m)
                      for m in state)
                  or any(self.scheduling_conflicted(state, (m//(len(self.teams)//2)), m, state[m])
                      for m in range(len(state)))
                  or self.final_match_conflicted(state)
                  )


    def value(self, state):
        return 0  # This is not an optimization problem, so value is not relevant

    def path_cost(self, c, state1, action, state2):
        return c + 1  # Cost is incremented by 1 for each step

<>:68: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:68: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-274-db02420d27d4>:68: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if state[-1] is (None, None):


In [ ]:
six_nations_with_final_match_constraint = NNationsTournamentWithFinalMatchConstraint(("IT", "FR", "EN", "WA", "SC", "EI"))
four_nations_with_final_match_constraint = NNationsTournamentWithFinalMatchConstraint(("IT", "WA", "EN", "FR"))

In [ ]:
%%timeit
depth_first_tree_search(four_nations_with_final_match_constraint)

33.4 ms ± 679 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
print(depth_first_tree_search(four_nations_with_final_match_constraint))

<Node (('FR', 'WA'), ('EN', 'IT'), ('EN', 'WA'), ('FR', 'IT'), ('FR', 'EN'), ('WA', 'IT'))>


In [ ]:
%%timeit
breadth_first_tree_search(four_nations_with_final_match_constraint)

110 ms ± 8.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
print(breadth_first_tree_search(four_nations_with_final_match_constraint))

<Node (('IT', 'EN'), ('WA', 'FR'), ('IT', 'FR'), ('WA', 'EN'), ('IT', 'WA'), ('EN', 'FR'))>


In [ ]:
%%timeit
depth_first_tree_search(six_nations_with_final_match_constraint)

21 s ± 1.16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print(depth_first_tree_search(six_nations_with_final_match_constraint))

<Node (('EI', 'SC'), ('WA', 'EN'), ('FR', 'IT'), ('EI', 'WA'), ('SC', 'FR'), ('EN', 'IT'), ('EI', 'EN'), ('WA', 'FR'), ('SC', 'IT'), ('SC', 'EN'), ('EI', 'FR'), ('WA', 'IT'), ('SC', 'WA'), ('EN', 'FR'), ('EI', 'IT'))>


# Implementation with Constraint 3

In [ ]:
class NNationsTournamentWithLimitedLocationConstraint(Problem):
    def __init__(self, teams):
        self.teams = teams
        self.matches = self.generate_matches(self.teams)
        self.initial = tuple([(None, None)] * ((len(self.matches)//2)))

        Problem.__init__(self, self.initial)

    def generate_matches(self, teams):
        matches = []
        for i in range(len(teams) - 1):
            for j in range(i + 1, len(teams)):
                matches.append((teams[i], teams[j]))
                matches.append((teams[j], teams[i]))
        return matches

    def actions(self, state):
        if state[-1] != (None, None):
            return []
        else:
            m_index = state.index((None, None))
            round = m_index // (len(self.teams)//2)
            return [m for m in self.matches
                    if not (self.scheduling_conflicted(state, round, m_index, m)
                    or self.asymmetry_conflicted(state, m_index, m)
                    or self.limited_location_conflicted(state, round, m)
                    )
                    ]

    def scheduling_conflicted(self, state, round, m_index, m):
      return any(self.scheduling_conflict(m, state[n])
                  for n in range(len(state))
                  if (state[n] != (None, None)
                  and ((n//(len(self.teams)//2)) == round)
                  and n != m_index))

    def scheduling_conflict(self, m, n):
      return (m[0] == n[0]
              or m[0] == n[1]
              or m[1] == n[0]
              or m[1] == n[1])

    def asymmetry_conflicted(self, state, m_index, m):
      return any(state[n] == m or state[n][::-1] == m
                 for n in range(len(state))
                 if n != m_index)

    def final_match_conflicted(self, state):
      return all(("EN", "FR") != state[n] and ("FR", "EN") != state[n]
                 for n in range(len(state)-(len(self.teams)//2), len(state))
                 if state[n] != (None, None)
                 )

    def limited_location_conflicted(self, state, round, m):
      return any(self.limited_location_conflict(m, state[n])
                  for n in range((round-1)*(len(self.teams)//2), round*(len(self.teams)//2))
                  if m[1] == state[n][1]
                )

    def limited_location_conflict(self, m, n):
      return (m[0] == "FR" and n[0] == "IT") or (m[0] == "IT" and n[0] == "FR")

    def result(self, state, action):
        new_match = state.index((None, None))
        new_state = list(state[:])
        new_state[new_match] = action
        return tuple(new_state)

    def goal_test(self, state):
      if state[-1] is (None, None):
        return False
      return not (any(self.asymmetry_conflicted(state, state.index(m), m)
                      for m in state)
                  or any(self.scheduling_conflicted(state, (m//(len(self.teams)//2)), m, state[m])
                      for m in range(len(state)))
                  or any(self.limited_location_conflicted(state, (m//(len(self.teams)//2)), state[m])
                          for m in range(len(state)))
                  or self.final_match_conflicted(state)
                  )

    def path_cost(self, c, state1, action, state2):
        return c + 1  # Cost is incremented by 1 for each step


<>:70: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:70: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-338-0e6305116c73>:70: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if state[-1] is (None, None):


In [ ]:
six_nations_with_limited_location_constraint = NNationsTournamentWithLimitedLocationConstraint(("IT", "FR", "EN", "WA", "SC", "EI"))
four_nations_with_limited_location_constraint = NNationsTournamentWithLimitedLocationConstraint(("IT", "WA", "EN", "FR"))

In [ ]:
%%timeit
depth_first_tree_search(four_nations_with_limited_location_constraint)

337 µs ± 7.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
print(depth_first_tree_search(four_nations_with_limited_location_constraint))

<Node (('FR', 'WA'), ('EN', 'IT'), ('EN', 'WA'), ('FR', 'IT'), ('FR', 'EN'), ('WA', 'IT'))>


In [ ]:
%%timeit
depth_first_tree_search(six_nations_with_limited_location_constraint)

3.2 ms ± 72 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
print(depth_first_tree_search(six_nations_with_limited_location_constraint))

<Node (('EI', 'SC'), ('WA', 'EN'), ('FR', 'IT'), ('EI', 'WA'), ('SC', 'FR'), ('EN', 'IT'), ('SC', 'WA'), ('EN', 'FR'), ('EI', 'IT'), ('EI', 'EN'), ('WA', 'FR'), ('SC', 'IT'), ('SC', 'EN'), ('EI', 'FR'), ('WA', 'IT'))>


In [ ]:
%%timeit
breadth_first_tree_search(four_nations_with_limited_location_constraint)

78.9 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print(breadth_first_tree_search(four_nations_with_limited_location_constraint))

<Node (('IT', 'WA'), ('EN', 'FR'), ('IT', 'EN'), ('WA', 'FR'), ('IT', 'FR'), ('WA', 'EN'))>


# Implementation with Constraints 2 & 3

Plus,


*   Naïve path cost: i.e. cost is incremented by 1 each step
*   Heuristic: Number of conflicting match placements for a given node



In [37]:
class NNationsTournamentWithConstraints(Problem):
    """The problem of organizing a rugby tournament with N teams.  A state is
    represented as an N-tuple array, where a value of (A, B) at position x means
    team A is playing against team B at home in the (x%0.5*|teams|+ 1)-th match
    of the (x//0.5*|teams|+ 1)-th round. Empty slots are represented by
    (None, None). Slots are filled left to right.
    """
    def __init__(self, teams):
        self.teams = teams
        self.matches = self.generate_matches(self.teams)
        self.initial = tuple([(None, None)] * ((len(self.matches)//2)))

        Problem.__init__(self, self.initial)

    def generate_matches(self, teams):
        """Generate all possible matches."""
        matches = []
        for i in range(len(teams) - 1):
            for j in range(i + 1, len(teams)):
                matches.append((teams[i], teams[j]))
                matches.append((teams[j], teams[i]))
        return matches

    def actions(self, state):
        """In the leftmost empty slot, try all non conflicting match placements."""
        if state[-1] != (None, None):
            return []
        else:
            m_index = state.index((None, None))
            round = m_index // (len(self.teams)//2)
            return [m for m in self.matches
                    if not (self.scheduling_conflicted(state, round, m_index, m)
                    or self.asymmetry_conflicted(state, m_index, m)
                    or self.limited_location_conflicted(state, round, m)
                    or self.final_match_conflicted(state, round, m_index, m)
                    )
                    ]

    def scheduling_conflicted(self, state, round, m_index, m):
      """Would placing a match m cause a scheduling conflict?"""
      return any(self.scheduling_conflict(m, state[n])
                  for n in range(len(state))
                  if (state[n] != (None, None)
                  and ((n//(len(self.teams)//2)) == round)
                  and n != m_index))

    def scheduling_conflict(self, m, n):
      """Would placing a match m and n cause a scheduling conflict?"""
      return (m[0] == n[0]
              or m[0] == n[1]
              or m[1] == n[0]
              or m[1] == n[1])

    def asymmetry_conflicted(self, state, m_index, m):
      """Would placing a match m cause a violation of the natural constraint of symmetry?"""
      return any(self.asymmetry_conflict(m, state[n])
                 for n in range(len(state))
                 if n != m_index)

    def asymmetry_conflict(self, m, n):
      """Would placing a match m and n conflict with the natural symmetry constraint?"""
      return (n == m or n[::-1] == m)

    def final_match_conflicted(self, state, round, m_index, m):
      """Would placing a match m cause a violation of the final match constraint?"""
      return any(self.final_match_conflict(state[m])
                for m in range(len(state)-(len(self.teams)//2))
                 if state[m] != (None, None)
                 )

    def final_match_conflict(self, m):
      """Check that placing a match m would cause a violation of the final match constraint."""
      return m == ("EN", "FR") or m[::-1] == ("EN", "FR")

    def limited_location_conflicted(self, state, round, m):
      """Would placing a new match m violate the limited location constraint?"""
      return any(self.limited_location_conflict(m, state[n])
                  for n in range(len(state))
                  if (m[1] == state[n][1]
                      and ((n in range((round-1)*(len(self.teams)//2), round*(len(self.teams)//2)))
                          or ((n//(len(self.teams)//2) == 0) and round == (len(self.matches)//2)//(len(self.teams)//2)))
                      )
                  )

    def limited_location_conflict(self, m, n):
      """Check that placing a match m and n would cause a limited location constraint."""
      return ((m[0] == "FR" and n[0] == "IT") or (m[0] == "IT" and n[0] == "FR"))

    def result(self, state, action):
        """Place the next match at a given slot."""
        new_match = state.index((None, None))
        new_state = list(state[:])
        new_state[new_match] = action
        return tuple(new_state)

    def goal_test(self, state):
      """Check if all matches are placed, and there are no conflicts."""
      if state[-1] == (None, None):
        return False
      return not (any(self.asymmetry_conflicted(state, state.index(m), m)
                      for m in state)
                  or any(self.scheduling_conflicted(state, (m//(len(self.teams)//2)), m, state[m])
                      for m in range(len(state)))
                  or any(self.final_match_conflicted(state, (m//(len(self.teams)//2)), m, state[m])
                      for m in range(len(state)-(len(self.teams)//2)))
                  or any(self.limited_location_conflicted(state, (m//(len(self.teams)//2)), state[m])
                          for m in range(len(state)))
                  )

    def path_cost(self, c, state1, action, state2):
        return c + 1  # Cost is incremented by 1 for each step

    def h(self, node):
        """Return number of conflicting match placements for a given node"""
        num_conflicts = 0
        for (m_index, m) in enumerate(node.state):
            if m_index in range(len(node.state)-(len(self.teams)//2)):
              num_conflicts += self.final_match_conflict(m)
            for (n_index, n) in enumerate(node.state):
                if n_index != m_index:
                    num_conflicts += self.asymmetry_conflict(m, n)
                if (n_index//(len(self.teams)//2)) == (m_index//(len(self.teams)//2)) and n_index != m_index: #Check if n, m are in the same round
                    num_conflicts += self.scheduling_conflict(m, n)
                if ((m[1] == n[1]) # Check if the same team is playing away.
                      and ((n_index in range(((m_index//(len(self.teams)//2))-1)*(len(self.teams)//2), (m_index//(len(self.teams)//2))*(len(self.teams)//2))) # Check if it's two consecutive matches
                          or ((n_index//(len(self.teams)//2)) == 0 # Check if it's consecutive matches due to symmetry (cf. natural constraint 1) ie n is in the first round, and m is in the .5*N(N-1)/(N/2)th round.
                                and (m_index//(len(self.teams)//2)) == (len(self.matches)//2)//(len(self.teams)//2)))
                      ):
                    num_conflicts += self.limited_location_conflict(m, n)

        return num_conflicts


In [38]:
six_nations_with_constraints = NNationsTournamentWithConstraints(("IT", "SC", "EN", "WA", "FR", "EI"))
four_nations_with_constraints = NNationsTournamentWithConstraints(("IT", "WA", "FR", "EN"))
eight_nations_with_constraints = NNationsTournamentWithConstraints(("IT", "SC", "EN", "WA", "FR", "EI", "ZA", "AU"))

In [57]:
%%timeit
depth_first_tree_search(four_nations_with_constraints)

576 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
for i in range(len(depth_first_tree_search(four_nations_with_constraints).solution())):
  print(f"Round {i//(len(four_nations_with_constraints.teams)//2) + 1}, Match {i%(len(four_nations_with_constraints.teams)//2)} : {depth_first_tree_search(four_nations_with_constraints).solution()[i]}")

Round 1, Match 0 : ('EN', 'WA')
Round 1, Match 1 : ('FR', 'IT')
Round 2, Match 0 : ('FR', 'WA')
Round 2, Match 1 : ('EN', 'IT')
Round 3, Match 0 : ('EN', 'FR')
Round 3, Match 1 : ('WA', 'IT')


In [36]:
%%timeit
depth_first_tree_search(six_nations_with_constraints)

4.3 ms ± 80.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
for i in range(len(depth_first_tree_search(six_nations_with_constraints).solution())):
  print(f"Round {i//(len(six_nations_with_constraints.teams)//2) + 1}, Match {i%(len(six_nations_with_constraints.teams)//2)+1} : {depth_first_tree_search(six_nations_with_constraints).solution()[i][0]} v. {depth_first_tree_search(six_nations_with_constraints).solution()[i][1]}")
for i in range(len(depth_first_tree_search(six_nations_with_constraints).solution())):
  print(f"Round {i//(len(six_nations_with_constraints.teams)//2) + 6}, Match {i%(len(six_nations_with_constraints.teams)//2)+1} : {depth_first_tree_search(six_nations_with_constraints).solution()[i][1]} v. {depth_first_tree_search(six_nations_with_constraints).solution()[i][0]}")

Round 1, Match 1 : EI v. FR
Round 1, Match 2 : WA v. EN
Round 1, Match 3 : SC v. IT
Round 2, Match 1 : EI v. WA
Round 2, Match 2 : FR v. SC
Round 2, Match 3 : EN v. IT
Round 3, Match 1 : FR v. WA
Round 3, Match 2 : EN v. SC
Round 3, Match 3 : EI v. IT
Round 4, Match 1 : EI v. EN
Round 4, Match 2 : WA v. SC
Round 4, Match 3 : FR v. IT
Round 5, Match 1 : FR v. EN
Round 5, Match 2 : EI v. SC
Round 5, Match 3 : WA v. IT
Round 6, Match 1 : FR v. EI
Round 6, Match 2 : EN v. WA
Round 6, Match 3 : IT v. SC
Round 7, Match 1 : WA v. EI
Round 7, Match 2 : SC v. FR
Round 7, Match 3 : IT v. EN
Round 8, Match 1 : WA v. FR
Round 8, Match 2 : SC v. EN
Round 8, Match 3 : IT v. EI
Round 9, Match 1 : EN v. EI
Round 9, Match 2 : SC v. WA
Round 9, Match 3 : IT v. FR
Round 10, Match 1 : EN v. FR
Round 10, Match 2 : SC v. EI
Round 10, Match 3 : IT v. WA


In [53]:
%%timeit
breadth_first_tree_search(four_nations_with_constraints)

41.5 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
print(breadth_first_tree_search(four_nations_with_constraints))

<Node (('IT', 'EN'), ('WA', 'FR'), ('IT', 'FR'), ('WA', 'EN'), ('IT', 'WA'), ('EN', 'FR'))>


In [ ]:
%%timeit
breadth_first_tree_search(six_nations_with_constraints)

In [ ]:
print(breadth_first_tree_search(six_nations_with_constraints))

In [39]:
import time
s = time.time()
for i in range(10):
  astar_search(four_nations_with_constraints)
e = time.time()
(e - s)/10

0.0032348155975341795

In [40]:
import time
s = time.time()
for i in range(10):
  astar_search(six_nations_with_constraints)
e = time.time()
(e - s)/10

0.08285388946533204

In [41]:
import time
s = time.time()
for i in range(10):
  astar_search(eight_nations_with_constraints)
e = time.time()
(e - s)/10

0.5174901247024536

In [46]:
print(astar_search(four_nations_with_constraints))

<Node (('EN', 'IT'), ('FR', 'WA'), ('EN', 'WA'), ('FR', 'IT'), ('EN', 'FR'), ('IT', 'WA'))>


In [59]:
%%timeit
astar_search(six_nations_with_constraints)

59.1 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
astar_search(eight_nations_with_constraints)

In [60]:
for i in range(len(astar_search(six_nations_with_constraints).solution())):
  print(f"Round {i//(len(six_nations_with_constraints.teams)//2) + 1}, Match {i%(len(six_nations_with_constraints.teams)//2)+1} : {astar_search(six_nations_with_constraints).solution()[i][0]} v. {astar_search(six_nations_with_constraints).solution()[i][1]}")

Round 1, Match 1 : EI v. EN
Round 1, Match 2 : FR v. IT
Round 1, Match 3 : SC v. WA
Round 2, Match 1 : EI v. FR
Round 2, Match 2 : EN v. SC
Round 2, Match 3 : IT v. WA
Round 3, Match 1 : EI v. IT
Round 3, Match 2 : EN v. WA
Round 3, Match 3 : FR v. SC
Round 4, Match 1 : EI v. SC
Round 4, Match 2 : EN v. IT
Round 4, Match 3 : FR v. WA
Round 5, Match 1 : EI v. WA
Round 5, Match 2 : EN v. FR
Round 5, Match 3 : IT v. SC
